In [ ]:
# https://www.smbtraining.com/member/weekly-options-income-machine/calendar

In [1]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchWindowException,WebDriverException,StaleElementReferenceException,NoSuchElementException
import datetime
import time as tm
import json
import pickle
import pprint
from td.client import TDClient
from td.exceptions import NotFndError,NotNulError
import traceback


with open('/home/cc/tdcred.json',"r") as read_file:
    cred = json.load(read_file)


COOKIE_PATH = "/home/cc/chrome-cookies.pkl"

TEST_PATH = None
TEST_STRIKE = None
TEST_PRICE_OFFSET = None

#TEST_PATH = 'file:///home/cc/options/process/python/MasterTrader/sample_page2/The%20Weekly%20Options%20Income%20Machine%20Program%20_%20Trader%20Training%20courses-SMB%20Capital-Stocks%20Options%20Trading%20tools.html'
#TEST_STRIKE = 4195
#TEST_PRICE_OFFSET = .50

STRIKE_SPREAD = 20
QUANTITY = 1

TRADE_SYMBOL = '$SPX.X'

MODIFY_ORDER_TRIES = 5
MODIFY_WAIT_TIME = 15

MAX_ORDERS = 3

MONITOR_TIME_START = datetime.time(6,10)
MONITOR_TIME_END = datetime.time(6,40)
#MONITOR_TIME_END = datetime.time(14,25)


with open('/home/cc/woim',"r") as read_file:
    info = json.load(read_file)
    
    
if TEST_PATH is not None:
    info['url'] = TEST_PATH
    
    
    
# cookie_login was used when cookies are loaded from already logged in session
# use manual log in now
 
def cookie_login_page(driver,info):

    driver.get(info['url1'])
    
    if TEST_PATH is not None:
        return


    cookies = pickle.load(open(COOKIE_PATH,"rb"))
    for cookie in cookies:
        driver.add_cookie(cookie)
    
    driver.get(info['url2'])

    
#info = { 'url': '/home/cc/Dropbox/sample_page/The Weekly Options Income Machine Program _ Trader Training courses-SMB Capital-Stocks Options Trading tools.html'}
#info = { 'url': 'sample_page/The Weekly Options Income Machine Program _ Trader Training courses-SMB Capital-Stocks Options Trading tools.html'}

  
#driver.get("https://google.co.in")

#need to load the page, in order to set the log in cookies, then load page again

def wait_for_element(func,id,tries):
    while True:
        try:
            form = func(id)
            return form
        
        except NoSuchElementException as e:
            tries -= 1
            if not tries:
                raise e
            tm.sleep(.1)
    

def manual_login(driver,cred):

    driver.get(cred['url1'])
    log_in = driver.find_element_by_xpath("// a[contains(text(),'Login')]")
    log_in.click()
    
    form = wait_for_element(driver.find_element_by_id,'user-login',10)


    edit_name = form.find_element_by_id("edit-name")
    edit_pass = form.find_element_by_id("edit-pass")
    edit_submit = form.find_element_by_id("edit-submit")
    edit_name.send_keys(cred['user'])
    edit_pass.send_keys(cred['pwd'])
    edit_submit.click()
    
    log_in = wait_for_element(driver.find_element_by_xpath,"// a[contains(text(),'Member Area')]",120)
    
    driver.get(cred['url2'])



    
def find_option_lowest_dte(session,symbol,spread_type,strike,date_today):
    
    print("find_option_lowest_dte: symbol= {} spread_type = {} strike = {} date_today = {}".format(symbol,spread_type,strike,date_today))
    
    opt_chain = {
    'symbol': symbol,
    #'contractType': spread_type,
    'optionType': 'S',
    'fromDate': date_today.strftime('%Y-%m-%d'),
    'afterDate': date_today.strftime("%Y-%m-%d"),
    'includeQuotes': True,
    'strike' : strike
    }
    
    maptype = {'put': 'putExpDateMap',
              'call': 'callExpDateMap'}
    
    option_chains = session.get_options_chain(option_chain=opt_chain)
    
    #print(len(option_chains))
    #return option_chains
    
    putcallExpDateMap = option_chains[maptype[spread_type]]
    
    lowest_dte = None
    lowest_key = None
    for key in putcallExpDateMap.keys():
        days_to_expire = int(key.split(':')[1])
        if lowest_dte is None or (days_to_expire >= 0 and days_to_expire < lowest_dte):
            lowest_dte = days_to_expire
            lowest_key = key
            
    print("lowest dte date:",lowest_key,lowest_dte)
            
    key = lowest_key

    key_date = lowest_key
    days_to_expire = lowest_dte
        
    #d = datetime.datetime.strptime(key_date,'%Y-%m-%d').date()
    #print('exp-date',key,key_date,d,expiration)
    #if d == expiration:
            #print("found one")
            
    info = putcallExpDateMap[key]
            
            #print('a')
            #print(info)
            
    for strike_entry in info.keys():
        
        print("fold:",float(strike),float(strike_entry))
                
        if (float(strike_entry) == float(strike)):
            for entry in info[strike_entry]:
                if "(AM)" in entry['description']:
                    continue
                #print("fold found:",entry['description'])
                #if '(PM)' in entry['description']:
                    #return entry
                return entry
                    
                
    return None

def get_leg_info(session,underlying_symbol,strike_type,sell_strike,buy_strike,date_today):
    sell_info = find_option_lowest_dte(session,underlying_symbol,strike_type,sell_strike,date_today)
    buy_info = find_option_lowest_dte(session,underlying_symbol,strike_type,buy_strike,date_today)
    
    return sell_info,buy_info


def create_order_payload(session,underlying_symbol,quantity,strike_type,sell_strike,buy_strike,date_today,mid_offset = 0):
    
    sell_info,buy_info = get_leg_info(session,underlying_symbol,strike_type,sell_strike,buy_strike,date_today)
    
    #print("sell_info")
    #pprint.pprint(sell_info)
    if sell_info is None:
        print("create_order_paload: no sell info")
        return None
    
    #print("buy_info")
    #pprint.pprint(buy_info)
    if buy_info is None:
        print("create_order_paload: no buy info")
        return None
        
    lowest_price = sell_info['bid'] - buy_info['ask']
                    
    mid_price = (sell_info['ask'] + sell_info['bid'])/2 - (buy_info['ask'] + buy_info['bid'])/2
    mid_price = round(mid_price,2)
        
    if TEST_PRICE_OFFSET is not None:
        mid_price += TEST_PRICE_OFFSET  #set a high price for now and adjust in TOS

                    
    print("natural mid price: ",mid_price, sell_info['ask'],sell_info['bid'],buy_info['ask'],buy_info['bid'])
    print("natural lowest price: ",lowest_price)
                    
    #mid_price = global_price
                    
    mid_price +=  mid_offset
                    
    if mid_price < lowest_price:                
        mid_price = lowest_price
            
    mid_price = round(mid_price,2)
    
    if mid_price <= 0:
        mid_price = .15
    
    print("using price:",mid_price)
    
    sell_limit_vertical_spread = {
        "session": "NORMAL",
        "duration": "DAY",
        "orderType": "NET_CREDIT",  #NET_DEBIT highest price to buy  NET_CREDIT highest price to sell.  (CC Note: I think should be lowest price to sell)
        "orderStrategyType": "SINGLE",
        "complexOrderStrategyType": "VERTICAL",
        "price": mid_price,
        "orderLegCollection": [
            {
                "orderLegType": "OPTION",
                "instrument": {
                    "symbol": sell_info['symbol'],
                    "assetType": "OPTION"
                },
                "instruction": "SELL_TO_OPEN",
                "quantity": quantity
            },
            {
                "orderLegType": "OPTION",
                "instrument": {
                    "symbol": buy_info['symbol'],
                    "assetType": "OPTION"
                },
                "instruction": "BUY_TO_OPEN",
                "quantity": quantity
            }
        ]
    }
    
    return sell_limit_vertical_spread

class MaxOrdersReachedError(Exception):
    pass


def cancel_trade(session,response_dict):
        try:
            cancel_response = session.cancel_order(cred['accountID'], response_dict['order_id'])
        except NotFndError:
            print("could not find while canceling",response_dict['order_id'])
            return False
        
        print("cancel response is:")
        pprint.pprint(cancel_response)
        
        #orders = session.get_orders_query(cred['accountID'])
        
        while True:
            print("searching for canceled order")
            
            try:
                order = session.get_orders(cred['accountID'], response_dict['order_id'])
            except NotFndError:
                print('could not find canceled order: ',response_dict['order_id'])
                return False
            
            if order['status'] == 'FILLED':
                print("order has been filled:",response_dict['order_id'])
                return False
            
            if order['status'] == 'CANCELED':
                break
                
            tm.sleep(.5)

                
        print("found order: ",order['orderId'])
        pprint.pprint(order)
        print("found order end")
        
        print("Order status is now: ",order['status'])
        
        return order


def manage_trade(session,quantity,strike_type,sell_strike,buy_strike,mid_offset=0):
    global global_price
    
    try_limit = MODIFY_ORDER_TRIES
    
    while(True):
        
        
        print("try num:",try_limit)
        order_dict = create_order_payload(session,TRADE_SYMBOL,quantity,strike_type,sell_strike,buy_strike,datetime.datetime.now().date(),mid_offset)
        
        try:
            response_dict = session.place_order(cred['accountID'],order_dict)
            print("response to new order")
            pprint.pprint(response_dict)
            order_accepted = True

        except NotNulError as e:
            print("manage_trade NotNullError:")
            print(e['error'])
            traceback.print_exc()
            order_accepted = False
        except:
            print( "manage_trade unexpected error:")
            traceback.print_exc()
            raise
            
            
        if order_accepted:
            tm.sleep(MODIFY_WAIT_TIME)
            
            cancel_response = cancel_trade(session,response_dict)
                
            if cancel_response is False:
                return
            
            quantity = cancel_response['quantity'] - cancel_response['filledQuantity']
            
            mid_offset -= .05

            try_limit -= 1
                    
        if try_limit<0:
            print("manage_trade, exceeded tries")
            return

        

                                                                
def get_TDSession():
    
    TDSession = TDClient(
        account_number=cred['accountID'],
        client_id = cred['client_id'],
        credentials_path = '/home/cc/Credentials/td_state.json',
        redirect_uri = "https://127.0.0.1"
    )

    TDSession.login()
    
    return TDSession

def test_td():
    TDSession = get_TDSession()
    
    msft_quotes = TDSession.get_quotes(instruments=['MSFT'])
    
    pprint.pprint(msft_quotes)
    
def show_all_prices(session,symbol,strike_type,sell_strike,spreads):

        prices = []
        
        for spread in spreads:
            buy_strike = sell_strike + (-spread if strike_type == 'put' else spread)

            order_dict = create_order_payload(session,symbol,1,strike_type,sell_strike,buy_strike,datetime.datetime.now().date())
            prices.append(order_dict['price'])
        
        for spread,price in zip(spreads,prices):
            buy_strike = sell_strike + (-spread if strike_type == 'put' else spread)
            print('show_all_prices:',sell_strike,buy_strike,price)
            
def execute_trade(option_info):
    
    
    TDSession = get_TDSession()
    
    strike_type = 'put' if option_info['put'] else 'call'
    
    sell_strike = option_info['strike']
    if TEST_STRIKE is not None:
        sell_strike = TEST_STRIKE

    buy_strike = sell_strike + (-STRIKE_SPREAD if option_info['put'] else STRIKE_SPREAD)
        
    
    manage_trade(TDSession,QUANTITY,strike_type,sell_strike,buy_strike)
    
    show_all_prices(TDsession,TRADE_SYMBOL,strike_type,sell_strike,[10,15,20])


def process_alerts(alert_texts):
    
    option_info = {
        'bullish':  None,
        'put' : None,
        'strike' : None
    }
    
    for alert in alert_texts:
        alert = alert.lower()
        
        if 'final' in alert: # this is todays final message
            
            print("final message")
            return False
        
        if 'chance' in alert:
            
            if 'bullish' in alert:
                option_info['bullish'] = True
                
            if 'bearish' in alert:
                option_info['bullish'] = False
                
        if 'suggesting' in alert:
            
            if 'put' in alert:
                option_info['put'] = True
            if 'call' in alert:
                option_info['put'] = False
            
            for word in alert.split():
                
                word = word.strip(',:.')

                try: option_info['strike'] = float(word)
                except:pass
    
        if 'not confirm' in alert:
            print("did not confirm")
            return False
        
        if option_info['bullish'] is not None and option_info['put'] is not None and option_info['strike'] is not None:
            
            if option_info['bullish'] != option_info['put']:
                print("mismatch")
                option_info['put'] = not option_info['put']
                
            print("trade confirmed:")
            
            print(option_info)
                        
            execute_trade(option_info)

            
            return False

    return True


def find_display(driver,info):
    #cookie_login(driver,info)
    manual_login(driver,info)
    game_plan = driver.find_element_by_id('frmGameplan')
    driver.switch_to.frame(game_plan)

    display = driver.find_element_by_id('display')
    return display
    
def find_alerts_and_times(display):
    alerts = display.find_elements_by_class_name('alert')
    times = display.find_elements_by_xpath('span')
    return alerts,times
    
def test_page(info):
    driver = webdriver.Chrome("/usr/local/bin/chromedriver")
    display = find_display(driver,info)
    find_alerts_and_times(display)
    tm.sleep(5)
    driver.close()


def monitor(driver,info,time_end):
    
    display = find_display(driver,info)
        
    date_text = display.find_element_by_xpath('h5').text
    print("monitor date:",date_text)
    
    stale_count = 0
    
    alert_texts = []

    while datetime.datetime.now().time() < MONITOR_TIME_END:
        
        alerts,times = find_alerts_and_times(display)
        
        for alert,time in zip(alerts,times):
                    
            try:
                alert_text = alert.text
                time_text = time.text
            except StaleElementReferenceException:
                print("Exception: Stale Element",stale_count)
                stale_count = stale_count + 1
                if stale_count == 5:
                    return
                else:
                    tm.sleep(.2)
                    break
            else:
                if alert_text not in alert_texts:
                    alert_texts.append(alert_text)
                
                    print('[',time_text,']')
                    print('[',alert_text,']')
                        
                    if not process_alerts(alert_texts):
                        return

                
    

#StaleElementReferenceException
#NoSuchWindowException
#WebDriverException

In [ ]:

test_td()
test_page(info)

print('Starting system at:',datetime.datetime.now())



while (True):
    
    if TEST_PATH is not None or TEST_STRIKE is not None:
        MONITOR_TIME_START = datetime.datetime.now().time()
        MONITOR_TIME_END = (datetime.datetime.combine(datetime.date.today(),MONITOR_TIME_START) + datetime.timedelta(seconds=90)).time()

    
    tm.sleep(.5)
    
    if datetime.datetime.now().time() <= MONITOR_TIME_START or datetime.datetime.now().time() > MONITOR_TIME_END:
        continue
        
    print('waking_up',datetime.datetime.now())
    
    driver = webdriver.Chrome("/usr/local/bin/chromedriver")

    try:
        
        monitor(driver,info,MONITOR_TIME_END)
        
    except NoSuchWindowException:
        print("no such window exception")
        
    except WebDriverException:
        print("web driver exception")
        
    except MaxOrdersReachedError:
        print("error max orders reached")
        
            
    print('back to sleep:',datetime.datetime.now())
    
    driver.quit()
    
    print("Waiting till:",MONITOR_TIME_END)
    while datetime.datetime.now().time() < MONITOR_TIME_END:
        tm.sleep(1)
    print("Wait finished",datetime.datetime.now())
        



Grabbing new access token...
{'MSFT': {'52WkHigh': 290.15,
          '52WkLow': 196.25,
          'askId': 'Q',
          'askPrice': 284.87,
          'askSize': 100,
          'assetMainType': 'EQUITY',
          'assetType': 'EQUITY',
          'bidId': 'P',
          'bidPrice': 284.55,
          'bidSize': 200,
          'bidTick': ' ',
          'closePrice': 286.22,
          'cusip': '594918104',
          'delayed': False,
          'description': 'Microsoft Corporation - Common Stock',
          'digits': 4,
          'divAmount': 2.24,
          'divDate': '2021-08-18 00:00:00.000',
          'divYield': 0.78,
          'exchange': 'q',
          'exchangeName': 'NASD',
          'highPrice': 288.618,
          'lastId': 'D',
          'lastPrice': 284.55,
          'lastSize': 0,
          'lowPrice': 286.08,
          'marginable': True,
          'mark': 284.87,
          'markChangeInDouble': -1.35,
          'markPercentChangeInDouble': -0.4717,
          'nAV': 0.0,
  

In [ ]:
#to set the cookies, load home below.  Log in, then pickle the cookies in next frame

from selenium import webdriver

def load_home():
    driver = webdriver.Chrome("/usr/local/bin/chromedriver")
    info = {}
    info['url'] = 'https://www.smbtraining.com/'
    driver.get(info['url'])
    return driver
#driver = load_home()

In [ ]:
import pickle
#only need to do once when logging in for first time.  This will save the login cookie which is loaded above
#pickle.dump(driver.get_cookies(), open(COOKIE_PATH,"wb"))

In [ ]:
import json
dct = {}
dct['url1'] = ''
dct['url2'] = ''
dct['user'] =  ''
dct['pwd'] = ''
#with open('/home/cc/woim','w') as outfile:
#    json.dump(dct,outfile)


In [ ]:
session = get_TDSession()


In [ ]:
strike_sell = 4180
strike_buy = 4170
quantity = 1

def get_info(session,strike_sell,strike_buy):

    sell_info,buy_info = get_leg_info(session,'$SPX.X','put',strike_sell,strike_buy,datetime.datetime.now().time())
    
    return sell_info,buy_info

print(datetime.datetime.now())
for i in range(1):
    sell_info,buy_info = get_info(session,strike_sell,strike_buy)
    print(sell_info['totalVolume'],datetime.datetime.fromtimestamp(sell_info['quoteTimeInLong']/1000))
    
print(datetime.datetime.now())

    
print(sell_info['ask'],sell_info['bid'],sell_info['theoreticalOptionValue'],sell_info['totalVolume'],sell_info['tradeTimeInLong'],sell_info['quoteTimeInLong'])
print(buy_info['ask'],buy_info['bid'],buy_info['theoreticalOptionValue'],buy_info['totalVolume'])

mid_price = (sell_info['ask'] + sell_info['bid'])/2 - (buy_info['ask'] + buy_info['bid'])/2
print("mid_price = ",mid_price)
theoretical_price = sell_info['theoreticalOptionValue'] - buy_info['theoreticalOptionValue']
print("theoretical_price = ",round(theoretical_price,2))

print(datetime.datetime.fromtimestamp(sell_info['quoteTimeInLong']/1000))

premium = quantity * mid_price
risk = quantity * (strike_sell - strike_buy)
print("premium = ",round(premium,2))
print("risk = ",risk)


In [ ]:
strike_sell = 4180
strike_buy = 4175
quantity = 4

def get_info(session,strike_sell,strike_buy):

    sell_info,buy_info = get_leg_info(session,'$SPX.X','put',strike_sell,strike_buy,datetime.datetime.now().time())
    
    return sell_info,buy_info

print(datetime.datetime.now())
for i in range(1):
    sell_info,buy_info = get_info(session,strike_sell,strike_buy)
    print(sell_info['totalVolume'],datetime.datetime.fromtimestamp(sell_info['quoteTimeInLong']/1000))
    
print(datetime.datetime.now())

    
print(sell_info['ask'],sell_info['bid'],sell_info['theoreticalOptionValue'],sell_info['totalVolume'],sell_info['tradeTimeInLong'],sell_info['quoteTimeInLong'])
print(buy_info['ask'],buy_info['bid'],buy_info['theoreticalOptionValue'],buy_info['totalVolume'])

mid_price = (sell_info['ask'] + sell_info['bid'])/2 - (buy_info['ask'] + buy_info['bid'])/2
print("mid_price = ",mid_price)
print("theoretical_price = ",sell_info['theoreticalOptionValue'] - buy_info['theoreticalOptionValue'])

print(datetime.datetime.fromtimestamp(sell_info['quoteTimeInLong']/1000))

premium = quantity * mid_price
risk = quantity * (strike_sell - strike_buy)
print("premium = ",premium)
print("risk = ",risk)


# manual log in

In [ ]:
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
import time as tm
import json

with open('/home/cc/woim',"r") as read_file:
    cred = json.load(read_file)

driver = webdriver.Chrome("/usr/local/bin/chromedriver")

def wait_for_element(func,id,tries):
    while True:
        try:
            form = func(id)
            return form
        
        except NoSuchElementException as e:
            tries -= 1
            if not tries:
                raise e
            tm.sleep(.1)
    

def manual_login(driver,cred):

    driver.get(cred['url1'])
    log_in = driver.find_element_by_xpath("// a[contains(text(),'Login')]")
    log_in.click()
    
    form = wait_for_element(driver.find_element_by_id,'user-login',10)


    edit_name = form.find_element_by_id("edit-name")
    edit_pass = form.find_element_by_id("edit-pass")
    edit_submit = form.find_element_by_id("edit-submit")
    edit_name.send_keys(cred['user'])
    edit_pass.send_keys(cred['pwd'])
    edit_submit.click()
    
    log_in = wait_for_element(driver.find_element_by_xpath,"// a[contains(text(),'Member Area')]",120)

manual_login(driver,cred)

driver.get(cred['url2'])




In [ ]:
driver = webdriver.Chrome("/usr/local/bin/chromedriver")
display = find_display(driver,info)

date_text = display.find_element_by_xpath('h5').text
print("monitor date:",date_text)


#alerts = display.find_elements_by_class_name('alert')
#times = display.find_elements_by_xpath('span')

alerts,times = find_alerts_and_times(display)




In [ ]:
alerts

In [ ]:
times

In [ ]:
len(alerts)

In [ ]:
len(times)

# show prices

In [ ]:
TDSession = get_TDSession()


In [ ]:
show_all_prices(TDSession,'$SPX.X','put',4290,[10,15,20])

show_all_prices(TDSession,'SPY','put',428,[1,2])


In [ ]:


put_spread_strike = 4160
width = 5

sell_limit_vertical_spread = create_order_payload(TDSession,'$SPX.X',1,'put',put_spread_strike,put_spread_strike-width,datetime.datetime.now().date(),mid_offset = 0)

print(sell_limit_vertical_spread['price'])


In [ ]:
width = 10

sell_limit_vertical_spread = create_order_payload(TDSession,'$SPX.X',1,'put',put_spread_strike,put_spread_strike-width,datetime.datetime.now().date(),mid_offset = 0)

print(sell_limit_vertical_spread['price'])


In [ ]:
width = 15

sell_limit_vertical_spread = create_order_payload(TDSession,'$SPX.X',1,'put',put_spread_strike,put_spread_strike-width,datetime.datetime.now().date(),mid_offset = 0)

print(sell_limit_vertical_spread['price'])


In [ ]:
width = 20

sell_limit_vertical_spread = create_order_payload(TDSession,'$SPX.X',1,'put',put_spread_strike,put_spread_strike-width,datetime.datetime.now().date(),mid_offset = 0)

print(sell_limit_vertical_spread['price'])


In [ ]:
datetime.datetime.now().date()

In [ ]:
    maptype = {'PUT': 'putExpDateMap',
              'CALL': 'callExpDateMap'}


In [ ]:
maptype['PUT']